In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from urllib.request import urlopen
import time

In [2]:
from selenium import webdriver

chrome_path = r'D:\\Telecom 1e anne\\Telecom Period 1\\Jupyter\\MODS203\\Project\\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
driver.get("https://www.kisskissbankbank.com/en/discover")

In [3]:
#function to get the html code from the requested url
def download_from_the_internet(url):
    try:
        return urlopen(url).read().decode('utf-8')
    except KeyboardInterrupt:
        raise
    except:
        return None

#function returns list of the tupels, where [0] – link to the project, [1] – category assigned to this project
def extract_links_from_html(html):  
    parser = BeautifulSoup(html)
#     parser = parser.find('div', attrs = {'class':'styles__StyledGrid-xsvioa-0 iBEewj kiss-BrowsingGrid'})
    parser = parser.find('div', attrs = {'class':'styles__StyledGrid-xsvioa-0'})
    return [(link.get('href'), link.find('div', attrs={'class':'k-CrowdfundingCard__subtitle__container'}).text) for link in parser.findAll('a')]

#functions return you list of the project links on the web-site
def get_all_projects_links(url, number):
    links = []
    for i in range(1,number):
        html = download_from_the_internet(url+'&page='+str(i))
#         print(html)
        links.extend(extract_links_from_html(html))
        time.sleep(5)
    return links

In [4]:
url = 'https://www.kisskissbankbank.com/en/discover?categories[music]=on&categories[adventure-and-sport]=on&categories[live-performance]=on&categories[film-and-video]=on&filter=all' #url with the filters applied
links = get_all_projects_links(url, 5)

In [7]:
url = 'https://www.kisskissbankbank.com/en/projects/effervescience-stressdefense'
html = requests.get(url)

In [9]:
soup = BeautifulSoup(html.text, 'html.parser')
comment = soup.findAll('li',attrs ={'data-test-id':'comments-tab'})
# comment
contri = soup.find_all('li',attrs ={'data-test-id':'backers-tab'})
# contri.get('href')

In [10]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="yes" data-react-helmet="true" name="mobile-web-app-capable"/><meta content="yes" data-react-helmet="true" name="apple-mobile-web-app-capable"/><meta content="KissKissBankBank" data-react-helmet="true" name="apple-mobile-web-app-title"/><meta content="#19b4fa" data-react-helmet="true" name="theme-color"/><meta content="#19b4fa" data-react-helmet="true" name="msapplication-navbutton-color"/><meta content="#19b4fa" data-react-helmet="true" name="apple-mobile-web-app-status-bar-style"/><meta content="/" data-react-helmet="true" name="msapplication-starturl"/><meta charset="utf-8" data-react-helmet="true"/><meta content="KissKissBankBank" data-react-helmet="true" property="author"/><meta content="KissKissBankBank" data-react-helmet="true" property="og:site_name"/><meta content="https://www.kisskissbankbank.com/en/projects/effervescience-stressdefense" data-react-helmet="true" property="og:url"/><meta content="en" data-react-helmet="tru

In [11]:
comment = soup.find('span', attrs ={'class':'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced'})
print(comment)
intro = soup.find_all('div', attrs ={'class':'marger__StyledMarger-sc-1qqifp5-0 kFDJHh intro__StyledInfoBlock-sc-1wfo5yh-5 lncbNE'})
com = soup.find('span', attrs ={'role':'alert'})

None


In [12]:
#function to create a list of projects for future dataframe 
def extract_data_from_html(html, category):
    soup = BeautifulSoup(html)
    title = soup.find('h1', attrs ={'data-test-id':'project-title'}).text
    infos = soup.find_all('div', attrs = {'class':'marger__StyledMarger-sc-1qqifp5-0 kFDJHh intro__StyledInfoBlock-sc-1wfo5yh-5 lncbNE'})
    '''
        Here we get the data about number of backers, amount of funded and target money.
        Also parsing percentage from the progress bar and calculating the 'is_successfull parameter'.
    '''
    for i in infos:
        if len(infos) == 2:
        #re.sub here is to clean the string of commas, symbols, etc.. Final output consitst only of digits. 
            funded = re.sub("[^0-9]", "",infos[1].find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-bold info__StyledBigText-lcf1ol-0 clfvNf'}).text)
            target = re.sub("[^0-9]", "",infos[1].find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-light info__StyledSmallText-lcf1ol-1 gDdVEG'}).text)
            backers = infos[0].find('div').text
        else:
            funded = re.sub("[^0-9]", "",infos[2].find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-bold info__StyledBigText-lcf1ol-0 clfvNf'}).text)
            target = re.sub("[^0-9]", "",infos[2].find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-light info__StyledSmallText-lcf1ol-1 gDdVEG'}).text)
            backers = infos[0].find('div').text

        percentage_fund = soup.find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-line-height-normal k-u-weight-regular progress__StyledText-ycznm1-1 iXebEz'}).text.split('\xa0')[0]
        if int(percentage_fund) >= 100: is_successfull = True
        else: is_successfull = False

    '''
        Here parsing the rewards for the backers. Prices is a list of rewards prices. 
        Number of rewards is the lenght of the prices list.
    '''
    #re.sub the same to leave only digits
    prices = [int(re.sub("[^0-9]", "",i.text)) for i in soup.find('div', attrs= {'id':'rewards'}).find_all('h2', attrs={'class':'title__StyledTitle-sc-46lshq-0 iYeFEL k-RewardCard__title k-u-margin-bottom-double k-u-margin-top-none'})]
    min_price = min(prices)
    max_price = max(prices)
    number_rewards = len(prices)
    
    thumbnail_type = 'https://www.kisskissbankbank.com/' + soup.find('img', attrs = {'class':'k-ButtonImage__img'}).get('src')
    
    num_news = '' #? driver.find_element_by_class_name('badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-HorizontalNav__badge')
    num_comments = soup.find('div', attrs ={'class':'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced'}).text #?
    num_contribution = '' #?
    '''
        Here parsing the owner web-page to get the number of the created projects.
    '''
    #going to the creator web-page and parsing the number of created projects
    html = download_from_the_internet('https://www.kisskissbankbank.com/'+soup.find('a', attrs = {'id':'owner_profile_button'}).get('href'))
    owner = BeautifulSoup(html)
    num_creater_projects = owner.find('span', attrs = {'class':'k-Badge k-HorizontalNav__badge'}).text.split('\n')[1]
    

    '''
        Here creating a dictionary with the obtained data and return in to the result list. 
        After all the data parsed, from this list of dictionaries a pandas DataFrame will be created.
    '''
    data = {'title':title, 
            'category':category,
            'is_successfull':is_successfull,
            'percentage_fund':percentage_fund,
            'funded':funded, 
            'target':target,  
            'backers':backers, 
            'thumbnail_type':thumbnail_type, 
            'number_rewards':number_rewards, 
            'min_price':min_price, 
            'max_price':max_price, 
            'num_news':num_news,
            'num_comments':num_comments, 
            'num_contribution':num_contribution,
            'num_creater_projects':num_creater_projects}
      
    return data

In [6]:
#function to create a list of projects for future dataframe 
def extract_data_from_html(url, html):
    data = []
    #Better to replace it also with selenium selector to obtain the page and go through the pagination
    soup = BeautifulSoup(html)
    content = soup.find('div', attrs = {'class':'styles__StyledGrid-xsvioa-0 iBEewj kiss-BrowsingGrid'})
    projects = content.find_all('a')
    for project in projects:
      #for every project creating a dictionary instance with the corresponding columns
      #parsing data from the card
        title = project.find('div', attrs={'class':'k-CrowdfundingCard__titleAndDesc'}).text
        category = project.find('div', attrs={'class':'k-CrowdfundingCard__subtitle__container'}).text
        infos = a.find('div', attrs={'class':'k-CrowdfundingCard__informations__infoContainer'})
        for i in infos:
            if len(infos) == 2:
                funded = infos[1].find('strong', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp'}).text
                target = infos[1].find('span', attrs={'class':'k-u-hidden@m-up'}).text
                backers = infos[0].find('strong', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp'}).text
            else:
                funded = infos[2].find('strong', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp'}).text
                target = infos[2].find('span', attrs={'class':'k-u-hidden@m-up'}).text
                backers = infos[0].find('strong', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp'}).text

        percentage_fund = project.find('div', attrs={'class':'k-CrowdfundingCard__progressBar'}).text.split()[0]
        if percentage_fund >= 100: is_successfull = True
        else: is_successfull = False
        #going to the project web-page
        #Here should be code to go to the project page
        tags = #?
        rewards = #?
        prices = #?
        min_price = #?
        max_price = #?
        in_2020 = #?
        thumbnail_type = #?
        number_rewards = #?
        num_news = #?
        num_comments = #?
        num_creater_projects = #?
        num_contributions = #?
        
        data.append({'title':title, 
                   'category':category,
                   'is_successfull':is_successfull,
                   'percentage_fund':percentage_fund,
                   'funded':funded, 
                   'target':target, 
                   'in_2020':in_2020, 
                   'backers':backers, 
                   'thumbnail_type':thumbnail_type, 
                   'number_rewards':number_rewards, 
                   'min_price':min_price, 
                   'max_price':max_price, 
                   'num_news':num_news,
                   'num_comments':num_comments, 
                   'num_contributions':num_contributions,
                   'num_creater_projects':num_creater_projects})
      
    return data

SyntaxError: invalid syntax (<ipython-input-6-cf2b5944bdb4>, line 29)

In [13]:
result = []
for link in links[:15]:
    url, category = link[0], link[1]
    html = download_from_the_internet(url)
    result.append(extract_data_from_html(html, category))
    time.sleep(1)

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
#recieve the html code of the web-page
html = download_from_the_internet('https://www.kisskissbankbank.com/en/projects/way-le-voyage-de-demain')
html